In [17]:
%pip install PyMuPDF pytesseract Pillow spacy
%pip install spacy
!python -m spacy download pl_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 38.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
import fitz  # PyMuPDF
from PIL import Image
import pytesseract
import spacy
import tqdm

# Function to convert PDF pages to images using PyMuPDF
def pdf_to_images(pdf_path):
    doc = fitz.open(pdf_path)
    images = []
    for page_num in range(doc.page_count):
        page = doc[page_num]
        image = page.get_pixmap()
        images.append(Image.frombytes("RGB", [image.width, image.height], image.samples))
    doc.close()
    return images

# Function to perform OCR using pytesseract on a list of images
def perform_ocr(images):
    ocr_results = []
    for img in images:
        text = pytesseract.image_to_string(img, lang='eng')
        ocr_results.append(text)
    return ocr_results

# Function to save OCR results to a text file
def save_to_text_file(ocr_results, output_path):
    with open(output_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write('\n'.join(ocr_results))

def extract_person_entities(text_path):
    nlp = spacy.load('pl_core_news_sm')
    with open(text_path, 'r', encoding='utf-8') as txt_file:
        text = txt_file.read()

    # Split the text into chunks of 500,000 characters
    chunk_size = 5000
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    persons = []
    for chunk in tqdm.tqdm(chunks):
        doc = nlp(chunk)
        persons.extend([ent.text for ent in doc.ents if ent.label_ == 'PERSON'])

    return persons

# Replace 'input.pdf' with the path to your PDF file
pdf_path = r'/content/34803_Spis_abonentow_sieci_telefonicznych_panstwowych_i__PDF.pdf'


# # Step 1: Convert PDF pages to images
# images = pdf_to_images(pdf_path)

# # Step 2: Perform OCR on the images
# ocr_results = perform_ocr(images)

# # Replace 'ocr_output.txt' with the desired output text file path
text_output_path = 'output.txt'

# # Step 3: Save OCR results to a text file
# save_to_text_file(ocr_results, text_output_path)

# Step 4: Extract PERSON entities using spaCy
persons = extract_person_entities(text_output_path)

with open('extracted_people.txt', 'w') as f:
  f.write('\n'.join(persons))


100%|██████████| 2254/2254 [09:07<00:00,  4.12it/s]


In [50]:
suffixes = [
    'ski ',
    'wicz ',
    'ieczny ',
    'yke ',
    'ska ',
    'rman ',
    'herpe ',
    'ritz ',
    'bieki ',
    'egiel ',
    'cki ',
    'adek ',
    'czek ',
    'ppe ',
    'piro '
]
def last_name(string):
  if not any([a in string for a in suffixes]):
    return None
  return string

In [51]:
skis = None
with open('/content/extracted_people_A.txt', 'r') as f:
  persons = f.read().splitlines()
  skis = set([last_name(a) for a in persons])

In [99]:
fns = ['Józef',
 'Stanisław',
 'Jakób',
 'Henryk',
 'Franciszek',
 'Władysław',
 'Zygmunt',
 'Stefan',
 'Kazimierz',
 'Leon',
 'Tadeusz',
 'Antoni',
 'Michał',
 'Paweł',
 'Jerzy',
 'Edward',
 'Marjan',
 'Roman',
 'Aleksander',
 'Feliks',
 'Piotr',
 'Ignacy',
 'Wacław',
 'Juljan',
 'Marja',
 'Adam',
 'Mieczysław',
 'Helena',
 'Andrzej',
 'Witold',
 'Edmund',
 'Bolesław',
 'Wojciech',
 'Włodzimierz',
 'Dawid',
 'Wiktor',
 'Zofja',
 'Abram',
 'Ludwik',
 'Teofil',
 'Konstanty',
 'Eugenjusz',
 'Adolf',
 'Tomasz',
 'Zdzisław',
 'Mojżesz',
 'Gustaw',
 'Izrael',
 'Stan',
 'Maksymiljan',
 'Juljusz',
 'Karol',
 'Rudolf',
 'Leonard',
 'Konrad',
 'Samuel',
 'Bernard',
 'Alfred',
 'Anna',
 'Aron',
 'Szymon',
 'Leopold',
 'Janusz',
 'Salomon',
 'Alojzy',
 'Czesław']

with open('/content/extracted_people_A.txt', 'r') as f:
  persons = f.read().splitlines()
  for person in persons:
    if any([name in fns for name in person.split(" ")]):
      skis.add(person)


In [268]:
clean = set()
import re
ignore = ['akcyjna','młyn','kredytowy','elektryczna','komitet','szkła','polska','związek','ajencja','agencja','spółka','fabryka','oszczędności','parafjalny','fabryka', 'spółdzelnia','sklep','hurtownia','handel','klub','agencja','notarjusz','hodowla','hotel','bank','posterunek','policji','urząd']
for name in skis:
  if name:
    pattern = "\x00"
    pattern2 = r""
    pattern3 = r"[a-zA-z]{1}\."
    pattern4 = r"[^\w\s]"
    name = re.sub(pattern, "", name)
    name = re.sub(pattern2, "", name)
    name = re.sub(pattern3, "", name)
    name = re.sub(pattern4, "", name)
    if not any([a in name.lower() for a in ignore]) and not re.search(r'\d+', name.lower()) and name.count(" ") >=1:
      if ' i ' not in name.lower():
        if all([len(a) >= 2 for a in name.split(" ")]):
          clean.add(name)
    else:
      continue
      #print(f"Ignore this one! {name}")

print(len(clean))
with open('final_person_names.txt', 'w') as f:
  f.write('\n'.join(clean))


21405


In [269]:
def split_names(name):
  last, *middle, first = name.split(" ")
  return last, middle, first

x = list(clean)

In [270]:
import csv

names = [split_names(i) for i in x]

people = []

for l, m, f in names:
  person = {'first':f, 'middle':' '.join(m), 'last':l}
  people.append(person)

with open('people_names.csv', 'w') as f:
  writer = csv.DictWriter(f, fieldnames=['first','middle','last'])
  writer.writeheader()
  writer.writerows(people)

In [265]:
from collections import Counter

first_names = [a if a.count('.')==0 and len(a) >= 3 else None for a in first_names]
top_first_names = Counter(first_names).most_common(300)

In [215]:
top_first = Counter(first_names).most_common(300)
top_first

[(None, 3720),
 ('Józef', 1993),
 ('Stanisław', 1037),
 ('Jakób', 806),
 ('Henryk', 664),
 ('Franciszek', 661),
 ('Władysław', 532),
 ('Zygmunt', 509),
 ('Stefan', 486),
 ('Kazimierz', 485),
 ('Leon', 484),
 ('Tadeusz', 430),
 ('Dawid', 410),
 ('Antoni', 400),
 ('Michał', 371),
 ('Paweł', 366),
 ('Samuel', 317),
 ('Jerzy', 301),
 ('Edward', 295),
 ('Marjan', 269),
 ('Adolf', 250),
 ('Salomon', 248),
 ('Roman', 244),
 ('Aleksander', 242),
 ('Szymon', 231),
 ('Feliks', 220),
 ('Gustaw', 213),
 ('Rudolf', 209),
 ('Piotr', 199),
 ('Ignacy', 196),
 ('Wacław', 187),
 ('Juljusz', 182),
 ('Juljan', 181),
 ('Aron', 172),
 ('Abram', 170),
 ('Izrael', 169),
 ('Alfred', 162),
 ('Marja', 160),
 ('Leopold', 152),
 ('Adam', 152),
 ('Mieczysław', 140),
 ('Mojżesz', 139),
 ('Helena', 136),
 ('Karol', 135),
 ('Jan', 134),
 ('Ludwik', 126),
 ('Bernard', 125),
 ('Andrzej', 122),
 ('Witold', 120),
 ('Wiktor', 119),
 ('Edmund', 115),
 ('Bolesław', 109),
 ('Zofja', 107),
 ('Maksymiljan', 107),
 ('Wojciech', 

In [220]:
first_names = set([a[0] if int(a[1])>= 25 else None for a in most_popular])

In [222]:
first_names

{'1',
 '1.',
 '11',
 '2',
 '25',
 '3',
 '6',
 '8',
 'A.',
 'Abram',
 'Adam',
 'Adolf',
 'Aleksander',
 'Alfred',
 'Alojzy',
 'Andrzej',
 'Anna',
 'Antoni',
 'Aron',
 'B.',
 'Bernard',
 'Bolesław',
 'Ch',
 'Czesław',
 'D.',
 'Dawid',
 'E.',
 'Edmund',
 'Edward',
 'Eugenjusz',
 'F.',
 'Feliks',
 'Fr',
 'Franciszek',
 'Gustaw',
 'H.',
 'Helena',
 'Henryk',
 'I.',
 'Ignacy',
 'Izrael',
 'J.',
 'Jakób',
 'Jan',
 'Janusz',
 'Jerzy',
 'Juljan',
 'Juljusz',
 'Józef',
 'K.',
 'Karol',
 'Kazimierz',
 'Konrad',
 'Konstanty',
 'L.',
 'Leon',
 'Leonard',
 'Leopold',
 'Ludwik',
 'M.',
 'Maksymiljan',
 'Marja',
 'Marjan',
 'Michał',
 'Mieczysław',
 'Mojżesz',
 'N.',
 'P.',
 'Paweł',
 'Piotr',
 'R.',
 'Roman',
 'Rudolf',
 'S.',
 'Salomon',
 'Samuel',
 'Ska',
 'St.',
 'Stan',
 'Stanisław',
 'Stefan',
 'Szymon',
 'T.',
 'Tad',
 'Tadeusz',
 'Teofil',
 'Tomasz',
 'W.',
 'Wacław',
 'Wiktor',
 'Witold',
 'Wojciech',
 'Wł',
 'Władysław',
 'Włodzimierz',
 'Z.',
 'Zdzisław',
 'Zofja',
 'Zygmunt',
 'i'}

In [98]:
to_find

['Józef',
 'Stanisław',
 'Jakób',
 'Henryk',
 'Franciszek',
 'Władysław',
 'Zygmunt',
 'Stefan',
 'Kazimierz',
 'Leon',
 'Tadeusz',
 'Antoni',
 'Michał',
 'Paweł',
 'Jerzy',
 'Edward',
 'Marjan',
 'Roman',
 'Aleksander',
 'Feliks',
 'Piotr',
 'Ignacy',
 'Wacław',
 'Juljan',
 'Marja',
 'Adam',
 'Mieczysław',
 'Helena',
 'Andrzej',
 'Witold',
 'Edmund',
 'Bolesław',
 'Wojciech',
 'Włodzimierz',
 'Dawid',
 'Wiktor',
 'Zofja',
 'Abram',
 'Ludwik',
 'Teofil',
 'Konstanty',
 'Eugenjusz',
 'Adolf',
 'Tomasz',
 'Zdzisław',
 'Mojżesz',
 'Gustaw',
 'Izrael',
 'Stan',
 'Maksymiljan',
 'Juljusz',
 'Karol',
 'Rudolf',
 'Leonard',
 'Konrad',
 'Samuel',
 'Bernard',
 'Alfred',
 'Anna',
 'Aron',
 'Szymon',
 'Leopold',
 'Janusz',
 'Salomon',
 'Alojzy',
 'Czesław']

In [129]:
x = 'sadfadsdffsdasdf'

In [130]:
s = re.search('\d+', x)

In [131]:
s